In [1]:
import time
import numpy as np
import pandas as pd
from scipy.linalg import logm, expm

In [4]:
df = pd.read_csv("HFData5DJIA1Y.csv")
df = df[df["DT"] != "2022-11-25 13:05:00"]          #Some stocks contain an observation at 13:05 on 2022-11-25, while other stock do not. These observations are removed

data = df[["SYMBOL", "PRICE", "DATE"]]          #Select the relevant information from the dataset
data

/var/folders/js/sz7lv_rx4tz_t0fgb1g89k440000gn/T/ipykernel_83157/2689352571.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("HFData5DJIA1Y.csv")


,SYMBOL,PRICE,DATE
0,MMM,178.320,2022-01-03
1,MMM,178.310,2022-01-03
2,MMM,178.480,2022-01-03
3,MMM,178.210,2022-01-03
4,MMM,177.339,2022-01-03
...,...,...,...
593795,DIS,85.960,2022-12-30
593796,DIS,86.160,2022-12-30
593797,DIS,86.410,2022-12-30
593798,DIS,86.750,2022-12-30


In [5]:
dates = np.unique(data["DATE"])
stocks = np.unique(data["SYMBOL"])

lmRCorr = []         #Initialize a list for the realized correlation matrices for all dates
for date in dates:
    nobsDate = len(data[(data["SYMBOL"] == "AAPL") & (data["DATE"] == date)])

    mRCovDate = np.zeros((len(stocks), len(stocks)))           #Initialize an empty realized covariance matrix, which will be used to compute the realized correlation matrix
    mRAllStocksOnDate = np.zeros((nobsDate - 1, len(stocks)))           #Initialize a matrix of returns for all the stocks on the date at hand

    s = 0
    for stock in stocks:
        dfStockOnDate = data[(data["SYMBOL"] == stock) & (data["DATE"] == date)]
        vRStocksOnDate = np.diff(dfStockOnDate["PRICE"]) / dfStockOnDate["PRICE"][:-1]
        mRAllStocksOnDate[:, s] = vRStocksOnDate.reset_index(drop=True)         #Store the return vector in the column of the corresponding stock
        s += 1

    for j in range(nobsDate - 1):
        mRCovDate += np.outer(mRAllStocksOnDate[j, :].T, mRAllStocksOnDate[j, :])

    diagonalElements = np.diag(mRCovDate)
    mRVarDateNegSqrt = np.diag(1 / np.sqrt(diagonalElements))
    mRCorrDate = mRVarDateNegSqrt @ mRCovDate @ mRVarDateNegSqrt         #Compute the realized correlation matrix from the realized covariance matrix

    lmRCorr.append(mRCorrDate)
    print(date)         #Print the date to indicate how fast the process is going

2022-01-03
2022-01-04
2022-01-05
2022-01-06
2022-01-07
2022-01-10
2022-01-11
2022-01-12
2022-01-13
2022-01-14
2022-01-18
2022-01-19
2022-01-20
2022-01-21
2022-01-24
2022-01-25
2022-01-26
2022-01-27
2022-01-28
2022-01-31
2022-02-01
2022-02-02
2022-02-03
2022-02-04
2022-02-07
2022-02-08
2022-02-09
2022-02-10
2022-02-11
2022-02-14
2022-02-15
2022-02-16
2022-02-17
2022-02-18
2022-02-22
2022-02-23
2022-02-24
2022-02-25
2022-02-28
2022-03-01
2022-03-02
2022-03-03
2022-03-04
2022-03-07
2022-03-08
2022-03-09
2022-03-10
2022-03-11
2022-03-14
2022-03-15
2022-03-16
2022-03-17
2022-03-18
2022-03-21
2022-03-22
2022-03-23
2022-03-24
2022-03-25
2022-03-28
2022-03-29
2022-03-30
2022-03-31
2022-04-01
2022-04-04
2022-04-05
2022-04-06
2022-04-07
2022-04-08
2022-04-11
2022-04-12
2022-04-13
2022-04-14
2022-04-18
2022-04-19
2022-04-20
2022-04-21
2022-04-22
2022-04-25
2022-04-26
2022-04-27
2022-04-28
2022-04-29
2022-05-02
2022-05-03
2022-05-04
2022-05-05
2022-05-06
2022-05-09
2022-05-10
2022-05-11
2022-05-12

In [6]:
lmRCorr[100]

array([[ 1.        , -0.11056264,  0.58443365,  0.6041791 ,  0.26992307,
         0.48318162,  0.31209116,  0.13913005,  0.5089685 ,  0.37609789,
         0.51141199,  0.3124681 ,  0.46608202,  0.28338028,  0.49676752,
        -0.10434561,  0.4671219 ,  0.2623506 ,  0.24584977,  0.32778031,
        -0.02191959,  0.70410846,  0.43935358, -0.0570343 , -0.03838748,
         0.14087621,  0.41528917,  0.00851404,  0.10729248,  0.07185458],
       [-0.11056264,  1.        , -0.21022101, -0.22247217,  0.05453226,
        -0.15940957,  0.2781662 ,  0.14911024, -0.14132001,  0.013475  ,
        -0.06030645, -0.03072626,  0.14672565,  0.409706  , -0.02432561,
         0.44628695, -0.07269866,  0.32884269,  0.27875352,  0.27367717,
         0.13918746,  0.08668413, -0.31285054,  0.23484711,  0.36038193,
         0.31946266, -0.18407205,  0.45165794,  0.30506142,  0.21304139],
       [ 0.58443365, -0.21022101,  1.        ,  0.81754558,  0.25093356,
         0.37566798,  0.30485258, -0.11029125,  0

In [7]:
stocks

array(['AAPL', 'AMGN', 'AXP', 'BA', 'CAT', 'CRM', 'CSCO', 'CVX', 'DIS',
       'DOW', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'KO', 'MCD',
       'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'V', 'VZ', 'WBA',
       'WMT'], dtype=object)

In [8]:
startTime = time.time()

lgammamRCorr = []
for i in range(len(lmRCorr)):
    logmCorr = logm(lmRCorr[i])           #Take the matrix logarithm
    vLowerDiagonal = logmCorr[np.tril_indices(logmCorr.shape[0], k=-1)]           #Take the lower diagonal entries
    lgammamRCorr.append(vLowerDiagonal)

timeSpent = time.time() - startTime
timeSpent

0.5230021476745605

In [9]:
lgammamRCorr[100]          #Length should be (30 x 29) / 2 = 435

array([-1.30023613e-01,  3.42167966e-01, -2.40096585e-01,  2.56409026e-01,
       -1.13046257e-01,  6.97873169e-01,  2.31590603e-02,  1.47277509e-02,
       -5.81656974e-02,  1.43596986e-01,  2.30661169e-01, -1.74243442e-01,
        1.08777567e-01,  3.48985318e-01,  4.20362219e-01,  5.62453441e-02,
        2.41771758e-01,  2.16351554e-01, -7.01900394e-02, -2.91291252e-01,
        2.65054705e-01,  1.61283668e-01, -3.58044254e-02, -6.83820001e-02,
        1.60756746e-01,  1.09139236e-01,  2.77564620e-01,  1.27753019e-01,
        1.73658829e-01,  5.43966444e-03,  5.24540524e-01,  3.89713884e-01,
       -1.18115275e-02,  2.48810079e-01,  1.88796129e-01, -2.71034897e-01,
       -6.80455865e-02, -7.96974571e-02,  3.87744752e-01,  1.04991038e-01,
       -4.49506757e-02, -1.96478090e-01, -8.97444499e-03,  1.40094787e-01,
        6.07481215e-02,  2.76049779e-01, -6.22236111e-03,  1.90484633e-01,
        4.19004584e-01, -1.73805786e-01,  1.89959051e-01,  1.70116664e-01,
        1.87706143e-01,  

In [10]:
def AlgorithmDiagonal(A, nStocks):
    x = np.random.normal(0, 1, size=nStocks)
    np.fill_diagonal(A, x)

    eps = 100
    while(eps > 10e-15):
        xnew = x - np.log(np.diag(expm(A)))

        eps = abs(sum(xnew - x))
        x = xnew
        np.fill_diagonal(A, x)

    return x

In [11]:
startTime2 = time.time()

nStocks = len(stocks)

lmRCorrMappedBack = []
for i in range(len(lgammamRCorr)):
    A = np.zeros((nStocks, nStocks))
    A[np.tril_indices(nStocks, k=-1)] = lgammamRCorr[i]
    A = A + A.T

    x = AlgorithmDiagonal(A, nStocks)
    np.fill_diagonal(A, x)

    lmRCorrMappedBack.append(expm(A))

timeSpent2 = time.time() - startTime2
timeSpent2

0.7294199466705322

In [12]:
error = 0
for i in range(len(lmRCorrMappedBack)):
    error += np.sum(np.abs(lmRCorrMappedBack[i] - lmRCorr[i]))

error

0.00019573505987188644

In [13]:
lmRCorrMappedBack[100]

array([[ 1.        , -0.11056264,  0.58443365,  0.6041791 ,  0.26992307,
         0.48318162,  0.31209116,  0.13913005,  0.5089685 ,  0.37609789,
         0.51141199,  0.3124681 ,  0.46608202,  0.28338028,  0.49676752,
        -0.10434561,  0.4671219 ,  0.2623506 ,  0.24584977,  0.32778031,
        -0.02191959,  0.70410846,  0.43935358, -0.0570343 , -0.03838748,
         0.14087621,  0.41528917,  0.00851404,  0.10729248,  0.07185458],
       [-0.11056264,  1.        , -0.21022101, -0.22247217,  0.05453226,
        -0.15940957,  0.2781662 ,  0.14911024, -0.14132001,  0.013475  ,
        -0.06030645, -0.03072626,  0.14672565,  0.409706  , -0.02432561,
         0.44628695, -0.07269866,  0.32884269,  0.27875352,  0.27367717,
         0.13918746,  0.08668413, -0.31285054,  0.23484711,  0.36038193,
         0.31946266, -0.18407205,  0.45165794,  0.30506142,  0.21304139],
       [ 0.58443365, -0.21022101,  1.00000002,  0.81754558,  0.25093356,
         0.37566799,  0.30485258, -0.11029125,  0